In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [81]:
a = pd.read_csv('/content/sample_data/train.csv')
b = pd.read_csv('/content/sample_data/train.csv')
df = pd.concat([a,b])
df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10299 entries, 0 to 2946
Columns: 563 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), int64(1), object(1)
memory usage: 44.3+ MB


In [64]:
df['Activity'].value_counts()

LAYING                1944
STANDING              1906
SITTING               1777
WALKING               1722
WALKING_UPSTAIRS      1544
WALKING_DOWNSTAIRS    1406
Name: Activity, dtype: int64

In [65]:
df.isnull().sum()

tBodyAcc-mean()-X       0
tBodyAcc-mean()-Y       0
tBodyAcc-mean()-Z       0
tBodyAcc-std()-X        0
tBodyAcc-std()-Y        0
                       ..
angle(X,gravityMean)    0
angle(Y,gravityMean)    0
angle(Z,gravityMean)    0
subject                 0
Activity                0
Length: 563, dtype: int64

In [66]:
df.shape

(10299, 563)

In [67]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# create independent and dependent columns
x = df.drop('Activity', axis=1)
y = df['Activity']

# LabelEncoding to target column
le = LabelEncoder()
le.fit_transform(y)

# split data into train and test (prevent data leakage)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=10)
print('x_train shape : ',x_train.shape)
print('x_test shape : ',x_test.shape)

# initialize and train logistic regression
lg = LogisticRegression(max_iter=1000)
lg.fit(x_train,y_train)
# Make prediction on test data
y_pred = lg.predict(x_test)
# compute accuracy score
accuracy_score = accuracy_score(y_test, y_pred)
print('accuracuy score',accuracy_score)

x_train shape :  (8239, 562)
x_test shape :  (2060, 562)
accuracuy score 0.9791262135922331


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Drop Duplicates Features

In [68]:
# drop duplicate columns
x_train = x_train.T.drop_duplicates().T
x_test = x_test.T.drop_duplicates().T

print(x_train.shape)
print(x_test.shape)

(8239, 541)
(2060, 541)


# Variance Threshold

In [69]:
from sklearn.feature_selection import VarianceThreshold
selection = VarianceThreshold(threshold=0.05)

selection.fit(x_train)

VarianceThreshold(threshold=0.05)

In [70]:
sum(selection.get_support())

347

In [71]:
columns = x_train.columns[selection.get_support()]

In [72]:
len(columns)

347

In [73]:
x_train = selection.transform(x_train)
x_test = selection.transform(x_test)

x_train = pd.DataFrame(x_train, columns=columns)
x_test = pd.DataFrame(x_test, columns=columns)

print(x_train.shape)
print(x_test.shape)

(8239, 347)
(2060, 347)


# Correlation

In [74]:
# Get the column names of the DataFrame
corr_metrics = x_train.corr()

In [75]:
# Create an empty list to keep track of columns to drop
columns_to_drop = []
# Loop over the columns
for i in range(len(columns)):
  for j in range(i+1, len(columns)):
    # Access the cell
    if corr_metrics.loc[columns[i],columns[j]] > 0.95:
      columns_to_drop.append(columns[j])
# print the length of drop columns (in this repeatation of name of columns may occured)
print(len(columns_to_drop))

1442


In [76]:
# for unique column name
columns_to_drop = set(columns_to_drop)
print(len(columns_to_drop))

195


In [77]:
# drop the columns
x_train.drop(columns=columns_to_drop,axis=1,inplace=True)
x_test.drop(columns=columns_to_drop,axis=1,inplace=True)
print(x_train.shape)
print(x_test.shape)

(8239, 152)
(2060, 152)


# ANOVA

In [78]:
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

# perform anova for classification case and get best 100 feature from data
sel = SelectKBest(f_classif, k=100)

# fit on data for understand data
sel.fit(x_train,y_train)

# selected best feature name
x_train.columns[sel.get_support()]

Index(['tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-max()-Z', 'tBodyAcc-min()-X', 'tBodyAcc-min()-Y',
       'tBodyAcc-min()-Z', 'tBodyAcc-entropy()-X', 'tBodyAcc-entropy()-Y',
       'tBodyAcc-entropy()-Z', 'tBodyAcc-arCoeff()-X,1',
       'tBodyAcc-arCoeff()-X,2', 'tBodyAcc-arCoeff()-X,3',
       'tBodyAcc-arCoeff()-Y,1', 'tBodyAcc-arCoeff()-Z,1',
       'tBodyAcc-correlation()-X,Y', 'tBodyAcc-correlation()-Y,Z',
       'tGravityAcc-mean()-X', 'tGravityAcc-mean()-Y', 'tGravityAcc-mean()-Z',
       'tGravityAcc-sma()', 'tGravityAcc-energy()-Y', 'tGravityAcc-energy()-Z',
       'tGravityAcc-entropy()-X', 'tGravityAcc-entropy()-Y',
       'tGravityAcc-arCoeff()-Y,1', 'tGravityAcc-arCoeff()-Y,2',
       'tGravityAcc-arCoeff()-Z,1', 'tGravityAcc-arCoeff()-Z,2',
       'tGravityAcc-correlation()-Y,Z', 'tBodyAccJerk-std()-Z',
       'tBodyAccJerk-min()-X', 'tBodyAccJerk-min()-Y', 'tBodyAccJerk-min()-Z',
       'tBodyAccJerk-entropy()-X', 'tBodyAccJerk-arCoeff

In [79]:
columns = x_train.columns[sel.get_support()]

In [80]:
# getting 100 best feature only
x_train = sel.transform(x_train)
x_test = sel.transform(x_test)

# convert to dataframe
x_train = pd.DataFrame(x_train, columns=columns)
x_test = pd.DataFrame(x_test, columns=columns)

print(x_train.shape)
print(x_test.shape)

(8239, 100)
(2060, 100)


# Chi-Square